# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
mongo

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database
collections = db.list_collection_names()

for collection in collections:
    print(collection)

establishments


In [18]:
# review the collections in our new database
collection = db.establishments

In [63]:
# review a document in the establishments collection
one_document = collection.find_one().keys()
one_document

dict_keys(['_id', 'FHRSID', 'ChangesByServerID', 'LocalAuthorityBusinessID', 'BusinessName', 'BusinessType', 'BusinessTypeID', 'AddressLine1', 'AddressLine2', 'AddressLine3', 'AddressLine4', 'PostCode', 'Phone', 'RatingValue', 'RatingKey', 'RatingDate', 'LocalAuthorityCode', 'LocalAuthorityName', 'LocalAuthorityWebSite', 'LocalAuthorityEmailAddress', 'scores', 'SchemeType', 'geocode', 'RightToReply', 'Distance', 'NewRatingPending', 'meta', 'links'])

In [20]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [29]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Halal",
    "AddressLine1": "123 Greenwich St",
    "AddressLine2": "",
    "AddressLine3": "",
    "AddressLine4": "",
    "PostCode": "45678",
    "Phone": "123-456-7890",
    "RatingValue": None,  
    "RatingKey": "AwaitingInspection",  
    "RatingDate": None,  
}

In [32]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant_data)


DuplicateKeyError: E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('658b0a3dd9eb62f311f5d384') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: uk_food.establishments index: _id_ dup key: { _id: ObjectId('658b0a3dd9eb62f311f5d384') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('658b0a3dd9eb62f311f5d384')}}

In [33]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('658b0a3dd9eb62f311f5d384'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal',
 'AddressLine1': '123 Greenwich St',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'PostCode': '45678',
 'Phone': '123-456-7890',
 'RatingValue': None,
 'RatingKey': 'AwaitingInspection',
 'RatingDate': None}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [44]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

establishments_collection = db.establishments

query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = establishments_collection.find_one(query, projection)

result

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [56]:
# Update the new restaurant with the correct BusinessTypeID
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type_result = establishments_collection.find_one(business_type_query, {"BusinessTypeID": 1})

new_restaurant_query = {"BusinessName": "Penang Flavours"}
new_restaurant_update = {
    "$set": {
        "BusinessTypeID": business_type_result["BusinessTypeID"],
        "BusinessType": "Restaurant/Cafe/Canteen"
        }
}

# Execute the update operation
update_result = establishments_collection.update_one(new_restaurant_query, new_restaurant_update)

updated_restaurant = establishments_collection.find_one(new_restaurant_query)




In [57]:
# Confirm that the new restaurant was updated
pprint(updated_restaurant)

{'AddressLine1': '123 Greenwich St',
 'AddressLine2': '',
 'AddressLine3': '',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Phone': '123-456-7890',
 'PostCode': '45678',
 'RatingDate': None,
 'RatingKey': 'AwaitingInspection',
 'RatingValue': None,
 '_id': ObjectId('658b0a3dd9eb62f311f5d384')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [59]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName': 'Dover'})

994

In [60]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_docs = establishments.delete_many({'LocalAuthorityName': 'Dover'})
delete_docs

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [61]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName': 'Dover'})

0

In [64]:
# Check that other documents remain with 'find_one'
establishments.count_documents({'LocalAuthorityName': 'Folkestone and Hythe'})

996

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [71]:
# Change the data type from String to Decimal for longitude and latitude
query = {
    'geocode.longitude': {'$type': 'string'},
    'geocode.latitude': {'$type': 'string'}
}

documents_to_update = establishments_collection.find(query)

for document in documents_to_update:
    longitude_double = float(document['geocode']['longitude'])
    latitude_double = float(document['geocode']['latitude'])

    update_query = {'_id': document['_id']}
    update_data = {
        '$set': {
            'geocode.longitude': longitude_double,
            'geocode.latitude': latitude_double
        }
    }

    establishments_collection.update_one(update_query, update_data)




Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

Use `update_many` to convert `RatingValue` to integer numbers.

In [72]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [73]:
# Change the data type from String to Integer for RatingValue
collection.update_many({}, {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}})

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [77]:
# Check that the coordinates and rating value are now numbers
establishments_collection = db.establishments

sample_document = establishments_collection.find_one()

if sample_document:
    longitude_type = type(sample_document.get('geocode', {}).get('longitude'))
    latitude_type = type(sample_document.get('geocode', {}).get('latitude'))
    rating_value_type = type(sample_document.get('RatingValue'))

    print(f"Data types after update:")
    print(f"Longitude: {longitude_type}")
    print(f"Latitude: {latitude_type}")
    print(f"RatingValue: {rating_value_type}")
else:
    print("No documents found")


Data types after update:
Longitude: <class 'float'>
Latitude: <class 'float'>
RatingValue: <class 'dict'>
